# Привлекательность и уровень заработной платы
## Постановка задачи
По 1260 опрошенным имеются следующие данные:

* `wage` - заработная плата за час работы, $;
* `exper` - опыт работы, лет;
* `educ` - образование, лет;
* `looks` - внешняя привлекательность, в баллах от 1 до 5;
* бинарные признаки: 
    * `female` - пол, 
    * `married` - семейное положение, 
    * `goodhlth` - состояние здоровья (хорошее/плохое), 
    * `union` - членство в профсоюзе, 
    * `black` - цвет кожи (белый/чёрный), 
    * `service` - занятость в сфере обслуживания (да/нет).

Требуется оценить влияние внешней привлекательности на уровень заработка с учётом всех остальных факторов.


## Импорты и загрузка данных

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import pandas as pd
import scipy.stats as st
import seaborn as sns

In [ ]:
data = pd.read_csv('./beauty.csv', delimiter=';')
data.head()

In [ ]:
data.columns

In [ ]:
new_cols = list(data.columns)
new_cols[0] = 'wage'
data.rename(columns=dict(zip(data.columns, new_cols)), inplace=True)
data.columns

Попарные диаграммы рассеяния всех количественных признаков: 

In [ ]:
sns.pairplot(data[['wage', 'exper', 'educ', 'looks']])

## Предобработка

Посмотрим на распределение оценок привлекательности: 

In [ ]:
_ = plt.hist(data['looks'])

В группах looks=1 и looks=5 слишком мало наблюдений. Превратим признак looks в категориальный и закодируем с помощью фиктивных переменных:



In [ ]:
data['below_avg'] = data['looks'].apply(lambda x: 1 if x < 3 else 0)
data['above_avg'] = data['looks'].apply(lambda x: 1 if x > 3 else 0)
looks = data.looks.copy()
data.drop(columns='looks', inplace=True)

data.head()

Распределение значений отклика:

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
data['wage'].plot.hist()
plt.xlabel('wage', fontsize=14)

plt.subplot(122)
np.log(data['wage']).plot.hist()
plt.xlabel('Log wage', fontsize=14)

Уберем выброс

In [ ]:
data = data[data['wage'] < 70]

## Модель 0

Просто построим линейную регрессию зарплаты от всех признаков

In [ ]:
feat_names = [f for f in data.columns if f not in ['wage']]
features = data[feat_names]
formula0 = ' '.join(['wage ~', 
                    ' + '.join([f for f in feat_names])])
formula0

In [ ]:
import statsmodels.api as sm
model0 = sm.OLS.from_formula(formula0, data).fit()
model0.summary()

#### Проанализируем ошибки

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
st.probplot(model0.resid, plot=plt)

plt.subplot(122)
model0.resid.plot.hist()
plt.xlabel('Residuals', fontsize=14)

Можем воспользоваться преобразованием Бокса-Кокса?

In [ ]:
max(data['wage']) / min(data['wage'])

$\frac{max y}{min y} \approx 40>10$, поэтому да.

Возьмём $λ=0$, то есть, будем строить регрессию логарифма отклика.

In [ ]:
log_wage = st.boxcox(data['wage'], 0)
old_columns = list(data.columns)
data1 = data.copy()
data1['log_wage'] = log_wage

In [ ]:
plt.hist(log_wage)
plt.xlabel('Log wage', fontsize=14)

## Модель 1

Построим линейную модель зависимости логарифма заработка по всем признакам. Проверим остатки на нормальность.

In [ ]:
features = data1[feat_names]
formula1 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in feat_names])])
print(formula1)

In [ ]:
model1 = sm.OLS.from_formula(formula1, data1).fit()
model1.summary()

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
st.probplot(model1.resid, plot=plt)

plt.subplot(122)
model1.resid.plot.hist()
_ = plt.xlabel('Residuals', fontsize=14)

Больше похоже на нормальное

### Проверка гипотез по остаткам

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
print(f'Нормальность ошибок: {st.shapiro(model1.resid)}')
print(f'Равенство распределений: {st.wilcoxon(model1.resid)}')
print(f'Гомоскедастичность: {het_breuschpagan(model1.resid, features)}')

### Рассмотрим зависимость от опыта и образования (самое очевидное)

In [ ]:
_ = sns.pairplot(pd.DataFrame({
    'exper': features['exper'], 
    'educ': features['educ'], 
    'residuals': model1.resid
}))

Есть ощущение, что `exper` распределены квадратично.

Взглянем более подробно:

In [ ]:
sns.residplot(features['exper'], 
              model1.resid,  
              lowess=True, 
              scatter_kws={'alpha': 0.5}, 
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
_ = plt.ylabel('residuals')

## Модель 2

Добавим в модель 1 квадрат опыта работы.

In [ ]:
exper2 = features['exper']**2
old_columns = list(data1.columns)
data2 = data1.copy()
data2['exper2'] = exper2
data2.head()

In [ ]:
feat_names += ['exper2']

In [ ]:
features = data2[feat_names]
formula2 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in feat_names])])
print(formula2)

In [ ]:
model2 = sm.OLS.from_formula(formula2, data2).fit()
model2.summary()

In [ ]:
gridsize = (2, 2)
fig = plt.figure(figsize=(16, 14))
ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
ax3 = plt.subplot2grid(gridsize, (1, 0))
ax4 = plt.subplot2grid(gridsize, (1, 1))
st.probplot(model2.resid, plot=ax1)

model2.resid.plot.hist(ax=ax2)
ax2.set_xlabel('Residuals')

sns.residplot(features['exper'], 
              model2.resid,
              lowess=True, 
              scatter_kws={'alpha': 0.5}, 
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
              ax=ax3)
ax3.set_ylabel('residuals')
sns.residplot(features['exper2'], 
              model2.resid,
              lowess=True, 
              scatter_kws={'alpha': 0.5}, 
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
              ax=ax4)
_ = ax4.set_ylabel('residuals')

Квадратичная зависимость ушла

### Проверка гипотез по остаткам

In [ ]:
print(f'Нормальность ошибок: {st.shapiro(model2.resid)}')
print(f'Равенство распределений: {st.wilcoxon(model2.resid)}')
print(f'Гомоскедастичность: {het_breuschpagan(model2.resid, features)}')

Остатки всё еще ненормальны и гетероскедастичны

Незначимые признаки: `goodhlth`, `black`, `married`, `above_avg`. Прежде, чем удалять лишние признаки, проверим, не входят ли они в значимые попарные взаимодействия:



In [ ]:
formula = [formula2]
formula

In [ ]:
for f1 in range(len(feat_names)):
    for f2 in range(f1 + 1, len(feat_names)):
        # ╰( ͡° ͜ʖ ͡° )-──☆*:・ﾟваш код
formula = ' + '.join(formula)
formula

In [ ]:
model_all = sm.OLS.from_formula(formula, data2).fit()

anova = sm.stats.anova_lm(model_all)
bad_names = []
for id, p in enumerate(anova['PR(>F)']):
    if p < 0.05:
        print(anova.iloc[id].name, p)
    else:
        bad_names.append(anova.iloc[id].name)

Визуальный анализ остатков не показывает никаких существенных особенностей: 

In [ ]:
_ = sns.pairplot(pd.DataFrame({
    'exper': features['exper'], 
    'exper2': features['exper2'],
    'educ': features['educ'], 
    'residuals': model_all.resid
}))

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
_ = st.probplot(model_all.resid, plot=plt)

plt.subplot(122)
_ = st.probplot(model2.resid, plot=plt)

# Модель 3
Удалим из модели 2 незначимые признаки (`goodhlth`, `black`, `married`) и добавим межфакторное взаимодействие пола и опыта работы, при этом оставим `above_avg`, так как мы хотим оценить влияние внешней привлекательности на зарплату

In [ ]:
bad_feats = ['goodhlth', 'black', 'married']
formula3 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in features if f not in bad_feats] + ['exper:female'])])
print(formula3)

In [ ]:
model3 = sm.OLS.from_formula(formula3, data=data2).fit()
model3.summary()

### Проверка гипотез по остаткам

In [ ]:
print(st.shapiro(model3.resid))
print(st.wilcoxon(model3.resid))

In [ ]:
print(st.shapiro(model2.resid))
print(st.wilcoxon(model3.resid))

Значимы все признаки, кроме индикатора привлекательности выше среднего.

Визуальный анализ остатков не показывает никаких существенных особенностей: 

In [ ]:
_ = sns.pairplot(pd.DataFrame({
    'exper': features['exper'], 
    'educ': features['educ'], 
    'residuals': model3.resid
}))

In [ ]:
st.probplot(model3.resid, plot=plt)

In [ ]:
model1.summary()

In [ ]:
model2.summary()

In [ ]:
model2.compare_lr_test(model3)

In [ ]:
model2.compare_f_test(model3)

# Модель 4
Попробуем оставить в модели 2 цвет кожи и семейное положение, чтобы добавить их взаимодействия с полом. Как и в модели 3, добавим взаимодействие пола с опытом работы, а состояние здоровья удалим.

In [ ]:
features.columns

In [ ]:
bad_feats = ['goodhlth']
add_feats = ['exper:female', 'black:female', 'married:female']
formula4 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in features if f not in bad_feats] + add_feats)])
print(formula4)

In [ ]:
model4 = sm.OLS.from_formula(formula4, data=data2).fit()
model4.summary()

### Проверка гипотез по остаткам

In [ ]:
print(st.shapiro(model4.resid))
print(st.wilcoxon(model4.resid))

In [ ]:
# _ = sns.pairplot(pd.concat([features[[ 'exper', 'educ', 'looks']], model4.resid],axis=1))
_ = sns.pairplot(pd.DataFrame({
    'exper': features['exper'], 
    'looks': looks,
    'residuals': model4.resid
}))

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
_ = st.probplot(model4.resid, plot=plt)

plt.subplot(122)
_ = st.probplot(model3.resid, plot=plt)

# Модель 5
В предыдущей модели семейное положение незначимо; посмотрим, можно ли удалить его

In [ ]:
features.columns

In [ ]:
bad_feats = ['goodhlth', 'married']
add_feats = ['exper:female', 'black:female', 'married:female']
formula5 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in features if f not in bad_feats] + add_feats)])
print(formula5)

In [ ]:
model5 = sm.OLS.from_formula(formula5, data=data2).fit()
model5.summary()

### Проверка гипотез по остаткам

In [ ]:
print(st.shapiro(model5.resid))
print(st.wilcoxon(model5.resid))

In [ ]:
print(st.shapiro(model4.resid))
print(st.wilcoxon(model4.resid))

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
_ = st.probplot(model4.resid, plot=plt)

plt.subplot(122)
_ = st.probplot(model5.resid, plot=plt)

Стоит вернуться к модели 4

# Расстояние Кука
Посмотрим на влиятельные наблюдения: 
    

In [ ]:
cook = model4.get_influence().summary_frame().loc[:,'cooks_d']

In [ ]:
plt.scatter(data2['log_wage'], cook)
plt.xlim((0,4))
plt.ylim((-0.01, 0.05))

Удалим наблюдения с расстоянием Кука больше 0.015 (порог выбран визуально) и перенастроим модель 4.

Сравним коэффициенты новой модели и модели 4:

In [ ]:
data3 = data2[cook < 0.015]
data3.shape

In [ ]:
model6 = sm.OLS.from_formula(formula4, data=data3).fit()
model6.summary()

In [ ]:
plt.figure(figsize=(16, 7))

plt.subplot(121)
plt.scatter(np.exp(model6.predict(data3)), data3['wage'])
plt.xlabel('wage', fontsize=14)
plt.ylabel('exp(predictions)', fontsize=14)

plt.subplot(122)
plt.scatter(model6.predict(data3), data3['log_wage'])
plt.xlabel('log wage', fontsize=14)
_ = plt.ylabel('predictions', fontsize=14)

## Итого

- Итоговая модель (№6) объясняет 43% вариации логарифма отклика
- Коэффициент при `below_avg`: -0.13 => человек, привлекательность которого ниже среднего, получает зарплату на 13% ниже, в среднем ($p=0.001$, 95%-й доверительный интервал: \[5,21\]%)
- Коэффициент при `above_avg`: -0.042 => человек, привлекательность которого выше среднего, примерно такую же, сколько и люди со средним уровнем привлекательности ($p =  0.884$, 95%-й доверительный интервал: \[-6, 6\]%), признак неинформативен